#**Two-Tower Neural Network for Candidate Generation**

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dot, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [4]:
customers = pd.read_csv('customers.csv')
policies = pd.read_csv('policies.csv')
interactions = pd.read_csv('interactions.csv')

In [5]:
# Merge customer and policy data into the interactions dataframe.
data = interactions.merge(customers, on='customer_id', how='left') \
                   .merge(policies, on='policy_id', how='left')

In [6]:
data['interaction_score'] = (
    (data['purchased'] * 5) +  # Strongest signal
    (data['abandoned_cart'] * 3) +  # Indicates strong intent
    (data['clicked'] * 2) +  # Clicks indicate interest
    (data['viewed_duration'] / 30) +  # Normalize by 30 seconds
    (data['comparison_count'] * 1)  # Weaker signal
)

In [7]:
data['label'] = data['interaction_score'] / data['interaction_score'].max()

In [8]:
customer_numeric_cols = ['age', 'policy_ownership_count', 'credit_score']
customer_categorical_cols = ['gender', 'income_bracket', 'employment_status', 'marital_status', 'location_city', 'preferred_policy_type']


In [9]:
# For the policy tower, we pick key numeric and categorical policy features.
policy_numeric_cols = ['sum_assured (INR)', 'premium_amount (INR)', 'policy_duration_years']
policy_categorical_cols = ['policy_type', 'risk_category', 'customer_target_group']

In [10]:
# Interaction features
interaction_numeric_cols = ['clicked', 'viewed_duration', 'comparison_count', 'abandoned_cart']

In [11]:
# Extract feature subsets
customer_features_df = data[customer_numeric_cols + customer_categorical_cols]
policy_features_df = data[policy_numeric_cols + policy_categorical_cols]
interaction_features_df = data[interaction_numeric_cols]

labels = data['label'].values

In [12]:
for column in ['sum_assured (INR)', 'premium_amount (INR)']:
    policy_features_df[column] = policy_features_df[column].str.replace(',', '').astype(float)

<ipython-input-12-299fafa69a5c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  policy_features_df[column] = policy_features_df[column].str.replace(',', '').astype(float)
<ipython-input-12-299fafa69a5c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  policy_features_df[column] = policy_features_df[column].str.replace(',', '').astype(float)


In [13]:
customer_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), customer_numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), customer_categorical_cols)
    ]
)

In [14]:
policy_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), policy_numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), policy_categorical_cols)
    ]
)

In [15]:
interaction_preprocessor = StandardScaler()

customer_features = customer_preprocessor.fit_transform(customer_features_df)
policy_features = policy_preprocessor.fit_transform(policy_features_df)
interaction_features = interaction_preprocessor.fit_transform(interaction_features_df)

In [16]:
X_cust_train, X_cust_test, X_policy_train, X_policy_test, X_interact_train, X_interact_test, y_train, y_test = train_test_split(
    customer_features, policy_features, interaction_features, labels, test_size=0.2, random_state=42
)

In [17]:
customer_input = Input(shape=(X_cust_train.shape[1],), name='customer_input')
policy_input = Input(shape=(X_policy_train.shape[1],), name='policy_input')
interaction_input = Input(shape=(X_interact_train.shape[1],), name='interaction_input')


In [18]:
# Customer tower
cust_dense = Dense(64, activation='relu')(customer_input)
cust_embed = Dense(32, activation='relu', name='customer_embedding')(cust_dense)

In [19]:
# Policy tower
policy_dense = Dense(64, activation='relu')(policy_input)
policy_embed = Dense(32, activation='relu', name='policy_embedding')(policy_dense)

In [20]:
# Interaction layer - This acts as an additional feature to influence the match score
interaction_dense = Dense(16, activation='relu')(interaction_input)

In [21]:
# Combine customer, policy, and interaction layers
combined = Concatenate()([cust_embed, policy_embed, interaction_dense])

# Final prediction layer
output = Dense(1, activation='sigmoid')(combined)

In [22]:
# Compile model
model = Model(inputs=[customer_input, policy_input, interaction_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ customer_input            │ (None, 36)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ policy_input (InputLayer) │ (None, 44)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │          2,368 │ customer_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64)             │          2,880 │ policy_input[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ interaction_input         │ (None, 4)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ customer_embedding        │ (None, 32)             │          2,080 │ dense[0][0]            │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ policy_embedding (Dense)  │ (None, 32)             │          2,080 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 16)             │             80 │ interaction_input[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 80)             │              0 │ customer_embedding[0]… │
│                           │                        │                │ policy_embedding[0][0… │
│                           │                        │                │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 1)              │             81 │ concatenate[0][0]      │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 9,569 (37.38 KB)

 Trainable params: 9,569 (37.38 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
history = model.fit(
    [X_cust_train, X_policy_train, X_interact_train], y_train,
    validation_split=0.1,
    epochs=10,
    batch_size=32
)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 306ms/step - accuracy: 0.1389 - loss: 0.6968 - val_accuracy: 0.1250 - val_loss: 0.7884
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 111ms/step - accuracy: 0.2331 - loss: 0.6655 - val_accuracy: 0.1250 - val_loss: 0.8032
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.2118 - loss: 0.6427 - val_accuracy: 0.1250 - val_loss: 0.8187
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.2196 - loss: 0.6351 - val_accuracy: 0.1250 - val_loss: 0.8312
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 105ms/step - accuracy: 0.2626 - loss: 0.6065 - val_accuracy: 0.1250 - val_loss: 0.8423
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.2352 - loss: 0.6180 - val_accuracy: 0.1250 - val_loss: 0.8444
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.2196 - loss: 0.6171 - val_accuracy: 0.1250 - val_loss: 0.8449
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.2509 - loss: 0.5889 - val_accuracy: 0.1250 - val_loss:

In [24]:
eval_results = model.evaluate([X_cust_test, X_policy_test, X_interact_test], y_test)
print(f"Test Loss: {eval_results[0]:.4f} | Test Accuracy: {eval_results[1]:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step - accuracy: 0.2000 - loss: 0.6414
Test Loss: 0.6414 | Test Accuracy: 0.2000


In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dot, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.utils import class_weight

# ------------------------
# 1. Data Loading & Merging
# ------------------------
customers = pd.read_csv('customers.csv')
policies = pd.read_csv('policies.csv')
interactions = pd.read_csv('interactions.csv')

data = interactions.merge(customers, on='customer_id', how='left') \
                   .merge(policies, on='policy_id', how='left')

# ------------------------
# 2. Label Creation & Analysis
# ------------------------
# Check purchase distribution (assumed to be a strong positive signal)
print("Purchase distribution:")
print(data['purchased'].value_counts())

# Use the 'purchased' flag as the label for binary classification.
data['label'] = data['purchased']

# Alternatively, if you prefer a composite score, you might do:
# data['interaction_score'] = (
#     (data['purchased'] * 5) +
#     (data['abandoned_cart'] * 3) +
#     (data['clicked'] * 2) +
#     (data['viewed_duration'] / 30) +
#     (data['comparison_count'] * 1)
# )
# # Use a threshold (for example, median) to binarize.
# threshold = data['interaction_score'].median()
# data['label'] = (data['interaction_score'] > threshold).astype(int)

# ------------------------
# 3. Feature Selection & Preprocessing
# ------------------------
# Define feature columns.
customer_numeric_cols = ['age', 'policy_ownership_count', 'credit_score']
customer_categorical_cols = ['gender', 'income_bracket', 'employment_status',
                             'marital_status', 'location_city', 'preferred_policy_type']

policy_numeric_cols = ['sum_assured (INR)', 'premium_amount (INR)', 'policy_duration_years']
policy_categorical_cols = ['policy_type', 'risk_category', 'customer_target_group']

interaction_numeric_cols = ['clicked', 'viewed_duration', 'comparison_count', 'abandoned_cart']

# Extract dataframes.
customer_features_df = data[customer_numeric_cols + customer_categorical_cols]
policy_features_df = data[policy_numeric_cols + policy_categorical_cols]
interaction_features_df = data[interaction_numeric_cols]

labels = data['label'].values

# Clean numeric columns for policies.
for column in ['sum_assured (INR)', 'premium_amount (INR)']:
    policy_features_df[column] = policy_features_df[column].str.replace(',', '').astype(float)

# Preprocessor for customer features.
customer_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), customer_numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), customer_categorical_cols)
    ]
)

# Preprocessor for policy features.
policy_preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), policy_numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), policy_categorical_cols)
    ]
)

# Standardize interaction features.
interaction_preprocessor = StandardScaler()

customer_features = customer_preprocessor.fit_transform(customer_features_df)
policy_features = policy_preprocessor.fit_transform(policy_features_df)
interaction_features = interaction_preprocessor.fit_transform(interaction_features_df)

# ------------------------
# 4. Train-Test Split
# ------------------------
X_cust_train, X_cust_test, X_policy_train, X_policy_test, X_interact_train, X_interact_test, y_train, y_test = train_test_split(
    customer_features, policy_features, interaction_features, labels, test_size=0.2, random_state=42
)

# ------------------------
# 5. Model Definition
# ------------------------
# Define model inputs.
customer_input = Input(shape=(X_cust_train.shape[1],), name='customer_input')
interaction_input = Input(shape=(X_interact_train.shape[1],), name='interaction_input')
policy_input = Input(shape=(X_policy_train.shape[1],), name='policy_input')

# Query Tower: Merge customer profile and interaction signals.
cust_dense = Dense(64, activation='relu')(customer_input)
cust_dense = Dropout(0.2)(cust_dense)
interact_dense = Dense(16, activation='relu')(interaction_input)
interact_dense = Dropout(0.2)(interact_dense)
query_concat = Concatenate()([cust_dense, interact_dense])
query_embed = Dense(32, activation='relu', name='query_embedding')(query_concat)

# Candidate Tower: Policy features.
policy_dense = Dense(64, activation='relu')(policy_input)
policy_dense = Dropout(0.2)(policy_dense)
policy_embed = Dense(32, activation='relu', name='policy_embedding')(policy_dense)

# Compute normalized dot product similarity.
similarity = Dot(axes=1, normalize=True)([query_embed, policy_embed])
# Optionally adjust scaling through an extra dense layer.
output = Dense(1, activation='sigmoid')(similarity)

model = Model(inputs=[customer_input, interaction_input, policy_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

# ------------------------
# 6. Training Enhancements
# ------------------------
# Early stopping to avoid overfitting.
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Compute class weights to address imbalance.
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train),
                                                  y=y_train)
class_weights_dict = dict(enumerate(class_weights))
print("Class weights:", class_weights_dict)

history = model.fit(
    [X_cust_train, X_interact_train, X_policy_train], y_train,
    validation_split=0.1,
    epochs=20,
    batch_size=32,
    callbacks=[early_stop],
    class_weight=class_weights_dict
)

# ------------------------
# 7. Model Evaluation
# ------------------------
eval_results = model.evaluate([X_cust_test, X_interact_test, X_policy_test], y_test)
print(f"Test Loss: {eval_results[0]:.4f} | Test Accuracy: {eval_results[1]:.4f}")


Purchase distribution:
purchased
0    88
1    12
Name: count, dtype: int64


<ipython-input-26-d120591fa53d>:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  policy_features_df[column] = policy_features_df[column].str.replace(',', '').astype(float)
<ipython-input-26-d120591fa53d>:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  policy_features_df[column] = policy_features_df[column].str.replace(',', '').astype(float)


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ customer_input            │ (None, 36)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ interaction_input         │ (None, 4)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 64)             │          2,368 │ customer_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 16)             │             80 │ interaction_input[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ policy_input (InputLayer) │ (None, 44)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 64)             │              0 │ dense_8[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 16)             │              0 │ dense_9[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 64)             │          2,880 │ policy_input[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_2             │ (None, 80)             │              0 │ dropout_3[0][0],       │
│ (Concatenate)             │                        │                │ dropout_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 64)             │              0 │ dense_10[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ query_embedding (Dense)   │ (None, 32)             │          2,592 │ concatenate_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ policy_embedding (Dense)  │ (None, 32)             │          2,080 │ dropout_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dot_1 (Dot)               │ (None, 1)              │              0 │ query_embedding[0][0], │
│                           │                        │                │ policy_embedding[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 1)              │              2 │ dot_1[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 10,002 (39.07 KB)

 Trainable params: 10,002 (39.07 KB)

 Non-trainable params: 0 (0.00 B)

Class weights: {0: 0.5555555555555556, 1: 5.0}
Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 527ms/step - accuracy: 0.8850 - loss: 0.7465 - val_accuracy: 0.8750 - val_loss: 0.6194
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - accuracy: 0.9045 - loss: 0.6697 - val_accuracy: 0.8750 - val_loss: 0.6188
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.9045 - loss: 0.6597 - val_accuracy: 0.8750 - val_loss: 0.6199
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.9045 - loss: 0.6623 - val_accuracy: 0.8750 - val_loss: 0.6212
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 124ms/step - accuracy: 0.8928 - loss: 0.6922 - val_accuracy: 0.8750 - val_loss: 0.6226
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 139ms/step - accuracy: 0.8000 - loss: 0.6391
Test Loss: 0.6391 | Test Accuracy: 0.8000


In [35]:
MODEL_PATH="twotower.h5"
model.save(MODEL_PATH)
print(f"Model saved at {MODEL_PATH}")

Model saved at twotower.h5
